In [0]:
#rm -r *

In [0]:
# Uncomment the line below and run this cell to get your data from github into colab (only runnable in colab, not ordinary jupyter notebook):
! git clone https://github.com/lkriener/music_generation.git && mv music_generation/* . && rm music_generation -r 

Cloning into 'music_generation'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (457/457), done.
remote: Total 500 (delta 68), reused 465 (delta 36), pack-reused 0
Receiving objects: 100% (500/500), 427.98 KiB | 14.27 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [0]:
# Uncomment line to install requirements
! pip install -r colab_requirements.txt

     |████████████████████████████████| 11.4MB 2.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ed/f6/fb/5d070524ecf7ba9ed201247a293c01945cfd7f840f8ef338c0
Successfully built pypianoroll


# Music generation using note-to-note LSTMs 
In this part, we will try to generate melodies from the same dataset (Bach Chorals) using recurrent neural networks (RNN) with long short term memory units (LSTM) as in the tutorial 08 of the class, based on the two following blog posts:

- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://towardsdatascience.com/writing-like-shakespeare-with-machine-learning-in-pytorch-d77f851d910c

The idea would be to extend the character-to-character LSTM network to midi files. We will also use another representation of the data, namely 'pianorolls', where a single melody is stored in a matrix with the x-axis for time and the y-axis for the pitch. This is one of the most frequent representations used, although it has some limitations. For instance, there is no way to differentiate between a long note (1 half-note) and 4 short-ones (4 eight-notes). However, it brings a considerable advantage compared to the previous used representations as it can store rhythm, depending on the chosen time-step. 

Moreover, each possible note is considered as a distinct element of a vocabulary. We will then use $N$ input nodes, where $N$ is the size of the vocabulary (i.e., the number of distinct notes, including silences). 
Thus, these pianorolls arrays will be converted into one-hot vectors containing the corresponding pitch (or silence, represented by 0) for each time step, and will be fed to the RNN network defined below. 

## Test of pypianoroll library
We will first use the pypianoroll library - https://salu133445.github.io/pypianoroll/ - to convert a midi file into a multitrack object, get the soprano track, transpose it to C major, create a new multitrack object out of it, and finally write it to a new midi file. 

In [0]:
import src.midi_utils as midi_utils
import pygame
from pypianoroll import Multitrack, Track
import pypianoroll
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1826816/45929032 bytes (4.0%)5660672/45929032 bytes (12.3%)9322496/45929032 bytes (20.3%)13180928/45929032 bytes (28.7%)17039360/45929032 bytes (37.1%)20701184/45929032 bytes (45.1%)24535040/45929032 bytes (53.4%)28352512/45929032 bytes (61.7%)32235520/45929032 bytes (70.2%)35971072/45929032 bytes (78.3%)39837696/45929032 bytes (86.7%)

In [0]:
try:
    home_dir
except NameError:
    home_dir = os.getcwd()

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir

In [0]:
from src.dataset_utils import TrackDataset, get_dataset_representation_from_tracks
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
label_scaler = StandardScaler()

# iterate over all midi files of folder
import glob
import os
import numpy as np

try:
    home_dir
except NameError:
    home_dir = os.getcwd()
    

In [0]:
def get_track_transposed(midi_filename, voice, beat_resolution, transpose = True):
    csv_text = midi_utils.load_to_csv(midi_filename)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # get multitrack object from midi 
    multitrack = pypianoroll.parse(midi_filename, beat_resolution=beat_resolution)
    # get the voice track 
    if transpose: 
        track = pypianoroll.transpose(multitrack.tracks[voice], -semitones)
    else:
        track = multitrack.tracks[voice]
    return track


def get_all_pianorolls(voice, home_dir, beat_resolution=4):
    '''
    Returns a large concatenated pianoroll from all Bach Chorals midi files
    :voice: 0 = soprano, 1 = alto, 2 = tenor, 3 = bass 
    :home_dir: home directory we extract midi files from
    :beat_resolution: minimal pianoroll time step. Default 4=sixteenth
    :return:
    '''
  
    list_pianorolls = []
    os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
  
    for midi_file in glob.glob("*.mid"):
        #print(midi_file)
        track = get_track_transposed(midi_file, voice, beat_resolution=beat_resolution)
        # get the flattened representation of pianoroll
        pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
        # add it to the global list of all tracks
        list_pianorolls.append(pianoroll_flattened)
        
    # convert into an array 
    all_pianorolls = np.concatenate(list_pianorolls)
    
    return all_pianorolls

def get_extremum_pitches(list_pianorolls):
    '''
    Return extremum pitches and pitch dictionnary of a given pitch sequence
    :list_pianorolls: list containing pianorolls sequence
    :return:
    '''
    list_min = []
    list_max = []
    
    for pianoroll in list_pianorolls:
        minimum  = np.min(pianoroll[np.nonzero(pianoroll)])
        maximum = np.max(pianoroll)
        list_min.append(minimum)
        list_max.append(maximum)
    global_lower = int(min(list_min))
    global_upper = int(max(list_max))
    n_notes = global_upper - global_lower + 2 
    
    return global_lower, global_upper, n_notes
        
        

In [0]:
# get lower and upper bounds 
voice = 0 
all_pianorolls = get_all_pianorolls(voice, home_dir, beat_resolution=4)
global_lower, global_upper, n_notes = get_extremum_pitches([all_pianorolls])
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

bwv245.5.mid
bwv2.6.mid
bwv266.mid
bwv364.mid
bwv183.5.mid
bwv397.mid
bwv197.10.mid
bwv337.mid
bwv187.7.mid
bwv277.mid
bwv339.mid
bwv6.6.mid
bwv197.7-a.mid
bwv310.mid
bwv270.mid
bwv410.mid
bwv154.3.mid
bwv248.35-3.mid
bwv245.37.mid
bwv3.6.mid
bwv9.7.mid
bwv144.3.mid
bwv153.1.mid
bwv256.mid
bwv14.5.mid
bwv244.40.mid
bwv36.8-2.mid
bwv113.8.mid
bwv101.7.mid
bwv330.mid
bwv380.mid
bwv178.7.mid
bwv227.11.mid
bwv159.5.mid
bwv258.mid
bwv293.mid
bwv77.6.mid
bwv87.7.mid
bwv56.5.mid
bwv360.mid
bwv120.6.mid
bwv36.4-2.mid
bwv417.mid
bwv177.5.mid
bwv381.mid
bwv373.mid
bwv157.5.mid
bwv267.mid
bwv322.mid
bwv114.7.mid
bwv325.mid
bwv244.25.mid
bwv284.mid
bwv190.7.mid
bwv244.62.mid
bwv416.mid
bwv117.4.mid
bwv406.mid
bwv280.mid
bwv89.6.mid
bwv283.mid
bwv245.15.mid
bwv194.6.mid
bwv72.6.mid
bwv422.mid
bwv430.mid
bwv145.5.mid
bwv245.26.mid
bwv84.5.mid
bwv180.7.mid
bwv248.5.mid
bwv345.mid
bwv65.7.mid
bwv365.mid
bwv370.mid
bwv92.9.mid
bwv248.53-5.mid
bwv399.mid
bwv341.mid
bwv434.mid
bwv420.mid
bwv313.mid
bwv37

In [0]:
print("Pianoroll sequence of soprano :")
print(all_pianorolls[0:100])

Pianoroll sequence of soprano :
[ 0  0  0  0 70 70 70  0 70 70 68 68 66 66 66 66 68 68 68 68 70 70 68 68
 66 66 66 66 65 65 65 65 63 63 63 63 70 70 70  0 70 70 70 70 68 68 68 68
 73 73 73 73 70 70 68 68 66 66 66 66 68 68 68 68 70 70 70  0 70 70 72 72
 73 73 73 73 75 75 77 77 78 78 78 78 77 77 77 77 75 75 75 75 74 74 74 74
 75 75 75  0]


In order to minimize the size of one-hot encode vectors, we restrict the pitches to values next to 0, using the minimum pitch found in the dataset. 

In [0]:
# scale pianoroll to 0 
print("Pianoroll sequence of soprano scaled to 0:")
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print(all_pianorolls_scaled[0:100])

Pianoroll sequence of soprano scaled to 0:
[ 0  0  0  0 14 14 14  0 14 14 12 12 10 10 10 10 12 12 12 12 14 14 12 12
 10 10 10 10  9  9  9  9  7  7  7  7 14 14 14  0 14 14 14 14 12 12 12 12
 17 17 17 17 14 14 12 12 10 10 10 10 12 12 12 12 14 14 14  0 14 14 16 16
 17 17 17 17 19 19 21 21 22 22 22 22 21 21 21 21 19 19 19 19 18 18 18 18
 19 19 19  0]


## Generation of a single voice

As a first attempt to generate a melody using LSTMs with pianorolls, we will simply try to generate a single voice (soprano, alto, tenor or bass), training the network on the whole midi file dataset concatenated into a long sequence of notes all translated to the same tonality (here, C Major or its relative A minor). The later operation will particularly facilitate learning on such a small dataset as all melodies will be rescaled to the same tonality, thus preventing abrupt changes among melody batches. 

In [0]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


### Define a LSTM network as for note-to-note melody generation
We now build a network to implement note-to-note melody generation using LSTMs units as well as drop-out of the output. This network is highly inspired from the Tutorial 08 of the class, by replacing characters by notes. 

In [0]:
# Declaring the model
class NoteRNN(nn.Module):
    
    def __init__(self, n_notes, n_hidden=256, n_layers=2,
                               drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.n_notes = n_notes 
        #define the LSTM
        self.lstm = nn.LSTM(self.n_notes, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.n_notes)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Training Code

Time for training! We declare a function, where we define an optimizer (Adam) and loss (cross entropy). Then, training and validation data are separated and the hidden state of the RNN is initaliazed. Looping over the training melody batches, we use the functions `get_pianoroll_batches` and `one_hot_encode_batch` in order to build batches and feed them to the network input. This function is also adapted to harmonization that will be defined below. 

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [0]:
# Declaring the train method
def train(net, data, data2=None, mode="melody_generation", epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 

        net: NoteRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    if mode == "harmonization":
        data2, val_data2 = data2[:val_idx], data2[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
        
    counter = 0
    n_notes = net.n_notes 
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        if mode == "melody_generation":
            batch_generator = midi_utils.get_pianoroll_batches(data, batch_size, seq_length)
        elif mode == "harmonization":
            batch_generator = midi_utils.get_pianoroll_batches_harmonization(data, data2, batch_size, seq_length)
        for x, y in batch_generator:
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = midi_utils.one_hot_encode_batch(x, n_notes)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                if mode == "melody_generation":
                    batch_generator_val = midi_utils.get_pianoroll_batches(val_data, batch_size, seq_length)
                elif mode == "harmonization":
                    batch_generator_val = midi_utils.get_pianoroll_batches_harmonization(val_data, val_data2, batch_size, seq_length)
                for x, y in batch_generator_val:
                    # One-hot encode our data and make them Torch tensors
                    x = midi_utils.one_hot_encode_batch(x, n_notes)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [204]:
# Define and print the net
n_hidden=128
n_layers=3

net = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 32
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, all_pianorolls_scaled, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

NoteRNN(
  (lstm): LSTM(50, 128, num_layers=3, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=128, out_features=50, bias=True)
)
Epoch: 2/20... Step: 50... Loss: 2.9264... Val Loss: 2.7865
Epoch: 4/20... Step: 100... Loss: 2.9130... Val Loss: 2.7793
Epoch: 6/20... Step: 150... Loss: 2.7743... Val Loss: 2.6484
Epoch: 8/20... Step: 200... Loss: 2.5160... Val Loss: 2.4050
Epoch: 10/20... Step: 250... Loss: 2.3035... Val Loss: 2.2073
Epoch: 12/20... Step: 300... Loss: 2.1197... Val Loss: 1.9854
Epoch: 14/20... Step: 350... Loss: 1.9405... Val Loss: 1.7959
Epoch: 16/20... Step: 400... Loss: 1.8122... Val Loss: 1.6625
Epoch: 18/20... Step: 450... Loss: 1.6288... Val Loss: 1.4857
Epoch: 20/20... Step: 500... Loss: 1.4512... Val Loss: 1.2859


### Test of the network: generation of single voice 
We will now test the trained network to predict potential melodies starting from a short sequence of notes. Here again, the functions `predict` and `sample` are highly inspired from the Tutorial 8, and slightly modified towards pianorolls data processing. 



In [0]:
""# Defining a method to generate the next character
def predict(net, note, h=None):
        ''' Given a note, predict the next note.
            Returns the predicted note and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[note]])
        x = midi_utils.one_hot_encode_batch(x, net.n_notes)
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)
        

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        note_range = np.arange(net.n_notes)
        # select the likely next note with some element of randomness
        p = p.numpy().squeeze()
        note = np.random.choice(note_range, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return note, h

In [0]:
# Declaring a method to generate new melody
def sample(net, size, prime=[10,10,12,12]):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime notes
    notes = [no for no in prime]
    h = net.init_hidden(1)
    for no in prime:
        note, h = predict(net, no, h)
    notes.append(note)
    
    # Now pass in the previous note and get a new one
    for ii in range(size):
        note, h = predict(net, notes[-1], h)
        notes.append(note)

    return np.array(notes)



def sample_harmonization(net, seq, prime):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    size = len(seq)
    
    # First off, run through the prime notes
    notes = [no for no in prime]
    h = net.init_hidden(1)
    for no in prime:
        note, h = predict(net, no, h)
    notes.append(note)
    
    # Now pass in the previous character and get a new one
    for ii in range(5,size):
        note, h = predict(net, seq[ii], h)
        notes.append(note)

    return np.array(notes)




The `NoteRNN` network outputs scaled pianorolls. We thus need to send back the obtained pitches to the associated voice range, using the function `unscale_pianoroll`. We then store the pianoroll to a track object, and to a multitrack object, which can directly be used to write a midi file. 

In [0]:
# Generating new melody
start = 72 - global_lower + 1 # starting with a C 
notes = sample(net, 200, prime=[start,start,start+2,start+2])
print("Generated sample :")
print(notes)
normal_pianoroll = midi_utils.unscale_pianoroll(notes, global_lower) # go back to the track range
print("Generated sample in the pitch range :")
print(normal_pianoroll)
# create a one_hot_pianoroll
one_hot_pianoroll = midi_utils.one_hot_encode_pianoroll(normal_pianoroll, 128)*90 # 90 for the velocity
# store it a in a track object
new_track = Track(pianoroll=one_hot_pianoroll, name='new track')
# create a multitrack made of the generated track object
new_multitrack = Multitrack(tracks=[new_track], tempo = 90, beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_track")
modified_midi_filename = home_dir + "/results/RNN_track.mid"

Generated sample :
[16 16 18 18 18 18 18 18 18 18  0 20 20 20 20 16 16 16 16 16 16 16 20 20
 20  0 20 20 21 21 23 23 23  0 23 23 20 23 18 18 18 18 20 20 20  0 20 20
 20 20 18 18 16 16 15 15 13 13 15 15 15 15 13 13 13  0 13 13 13  0 13 13
 13  0 13 13 13 13 11 11 11  0 11 11 11  0 11 11 11 11 16 16 16 16 18 18
 18 18 20 20 20 20 18 18 18 18 18 18 16 16 15 15 15 15 13 13 13 13 16 16
 15 15 13 13 13 13 15 15 15 15 16 16 16 16 15 15 15 15 13 13 13 13  0  0
  0  0 13 13 14  0 14 14 13 13  9  9  9  9  8  8  8  8 13 13 13 13 11 11
 11 11 16 16 16 16 13 13 15 15 16 16 16  0 16 16 16 16 15 15 15 15 13 13
  0 13 11 11 11 16 16 16 16 15 15 15 15]
Generated sample in the pitch range :
[72 72 74 74 74 74 74 74 74 74  0 76 76 76 76 72 72 72 72 72 72 72 76 76
 76  0 76 76 77 77 79 79 79  0 79 79 76 79 74 74 74 74 76 76 76  0 76 76
 76 76 74 74 72 72 71 71 69 69 71 71 71 71 69 69 69  0 69 69 69  0 69 69
 69  0 69 69 69 69 67 67 67  0 67 67 67  0 67 67 67 67 72 72 72 72 74 74
 74 74 76 76 76 76 74 74 7

In [0]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio
from midi2audio import FluidSynth
from IPython.display import Audio



In [0]:
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename, 'test.wav')
Audio("test.wav")

Output hidden; open in https://colab.research.google.com to view.

In [0]:

pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [0]:
pygame.mixer.music.stop()

error: ignored

## Train for harmonization

We observed above that it is possible to generate a coherent melody using a slightly modified version of the original character-to-character RNN. Here, we will investigate whether such a network can perform more complex task, such as harmonizing a given voice. For instance, from the soprano sequence, we would like to generate the associated alto voice. 

In [0]:
beat_resolution = 2

In [0]:

all_pianorolls_soprano = get_all_pianorolls(0, home_dir, beat_resolution=beat_resolution)
all_pianorolls_alto = get_all_pianorolls(1, home_dir, beat_resolution=beat_resolution)
all_pianorolls_tenor = get_all_pianorolls(2, home_dir, beat_resolution=beat_resolution)
all_pianorolls_bass = get_all_pianorolls(3, home_dir, beat_resolution=beat_resolution)

In [0]:
list_pianorolls = [all_pianorolls_soprano, all_pianorolls_alto, all_pianorolls_tenor, all_pianorolls_bass]
global_lower, global_upper, n_notes = get_extremum_pitches(list_pianorolls)


print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 35
Global upper note : 83
Number of notes : 50


We now rescale the pianorolls to values next to 0. 

In [0]:
# scale pianoroll to 0 
all_pianorolls_soprano = midi_utils.scale_pianoroll(all_pianorolls_soprano, global_lower)
all_pianorolls_alto = midi_utils.scale_pianoroll(all_pianorolls_alto, global_lower)
all_pianorolls_tenor = midi_utils.scale_pianoroll(all_pianorolls_tenor, global_lower)
all_pianorolls_bass = midi_utils.scale_pianoroll(all_pianorolls_bass, global_lower)


It is now time to train the network to perform harmonization. Setting the mode to `"harmonization"`, we will call the function `get_pianoroll_batches_harmonization` that generates one-hot vectors of both input and target melodies. 

In [253]:
# Define and print the net
n_hidden=512
n_layers=2

net_harmonization_alto = NoteRNN(n_notes, n_hidden, n_layers)
net_harmonization_tenor = NoteRNN(n_notes, n_hidden, n_layers)
net_harmonization_bass = NoteRNN(n_notes, n_hidden, n_layers)



# Declaring the hyperparameters
batch_size = 64
seq_length = 50
n_epochs = 100 # start smaller if you are just testing initial behavior

# train the model
train(net_harmonization_alto, data=all_pianorolls_soprano, data2=all_pianorolls_alto, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

Epoch: 5/100... Step: 50... Loss: 2.6837... Val Loss: 2.6952
Epoch: 9/100... Step: 100... Loss: 2.1628... Val Loss: 2.1865
Epoch: 13/100... Step: 150... Loss: 1.9556... Val Loss: 2.0641
Epoch: 17/100... Step: 200... Loss: 1.7638... Val Loss: 1.9552
Epoch: 21/100... Step: 250... Loss: 1.7696... Val Loss: 1.9305
Epoch: 25/100... Step: 300... Loss: 1.6271... Val Loss: 1.9085
Epoch: 30/100... Step: 350... Loss: 1.5377... Val Loss: 1.8743
Epoch: 34/100... Step: 400... Loss: 1.5111... Val Loss: 1.9149
Epoch: 38/100... Step: 450... Loss: 1.4121... Val Loss: 1.9774
Epoch: 42/100... Step: 500... Loss: 1.2328... Val Loss: 2.0196
Epoch: 46/100... Step: 550... Loss: 1.1948... Val Loss: 2.0242
Epoch: 50/100... Step: 600... Loss: 1.0025... Val Loss: 2.1039
Epoch: 55/100... Step: 650... Loss: 0.8604... Val Loss: 2.2634
Epoch: 59/100... Step: 700... Loss: 0.8123... Val Loss: 2.3946
Epoch: 63/100... Step: 750... Loss: 0.7413... Val Loss: 2.5258
Epoch: 67/100... Step: 800... Loss: 0.6590... Val Loss: 2.

In [254]:
train(net_harmonization_tenor, data=all_pianorolls_soprano, data2=all_pianorolls_tenor, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

Epoch: 5/100... Step: 50... Loss: 2.6854... Val Loss: 2.9172
Epoch: 9/100... Step: 100... Loss: 2.5042... Val Loss: 2.4858
Epoch: 13/100... Step: 150... Loss: 2.2193... Val Loss: 2.2892
Epoch: 17/100... Step: 200... Loss: 1.9316... Val Loss: 2.0811
Epoch: 21/100... Step: 250... Loss: 1.8708... Val Loss: 2.0714
Epoch: 25/100... Step: 300... Loss: 1.7279... Val Loss: 2.0192
Epoch: 30/100... Step: 350... Loss: 1.6773... Val Loss: 2.0358
Epoch: 34/100... Step: 400... Loss: 1.6837... Val Loss: 2.0381
Epoch: 38/100... Step: 450... Loss: 1.5756... Val Loss: 2.0743
Epoch: 42/100... Step: 500... Loss: 1.4114... Val Loss: 2.0881
Epoch: 46/100... Step: 550... Loss: 1.4110... Val Loss: 2.1512
Epoch: 50/100... Step: 600... Loss: 1.1396... Val Loss: 2.2130
Epoch: 55/100... Step: 650... Loss: 1.0974... Val Loss: 2.3089
Epoch: 59/100... Step: 700... Loss: 1.0607... Val Loss: 2.3965
Epoch: 63/100... Step: 750... Loss: 0.7752... Val Loss: 2.5673
Epoch: 67/100... Step: 800... Loss: 0.7034... Val Loss: 2.

In [255]:
train(net_harmonization_bass, data=all_pianorolls_soprano, data2=all_pianorolls_bass, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

Epoch: 5/100... Step: 50... Loss: 2.8779... Val Loss: 2.7923
Epoch: 9/100... Step: 100... Loss: 2.6281... Val Loss: 2.6543
Epoch: 13/100... Step: 150... Loss: 2.3573... Val Loss: 2.3237
Epoch: 17/100... Step: 200... Loss: 2.0670... Val Loss: 2.1591
Epoch: 21/100... Step: 250... Loss: 2.0083... Val Loss: 2.0996
Epoch: 25/100... Step: 300... Loss: 1.7172... Val Loss: 2.1159
Epoch: 30/100... Step: 350... Loss: 1.5050... Val Loss: 2.1937
Epoch: 34/100... Step: 400... Loss: 1.2510... Val Loss: 2.3346
Epoch: 38/100... Step: 450... Loss: 1.0879... Val Loss: 2.4346
Epoch: 42/100... Step: 500... Loss: 0.9572... Val Loss: 2.6733
Epoch: 46/100... Step: 550... Loss: 0.6880... Val Loss: 2.8188
Epoch: 50/100... Step: 600... Loss: 0.5870... Val Loss: 3.0196
Epoch: 55/100... Step: 650... Loss: 0.3647... Val Loss: 3.2446
Epoch: 59/100... Step: 700... Loss: 0.3840... Val Loss: 3.4572
Epoch: 63/100... Step: 750... Loss: 0.3125... Val Loss: 3.6780
Epoch: 67/100... Step: 800... Loss: 0.2377... Val Loss: 3.

In [0]:
 # Generating harmonization

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
# get one voice from the dataset 
#midi_filename = 'bwv368.mid'
midi_filename = 'bwv155.5.mid'
#midi_filename = 'pkgsc_azalea.mid'
soprano_track = get_track_transposed(midi_filename, 0, beat_resolution=beat_resolution, transpose=True)
alto_track_real = get_track_transposed(midi_filename, 1, beat_resolution=beat_resolution, transpose=True)
tenor_track_real = get_track_transposed(midi_filename, 2, beat_resolution=beat_resolution, transpose=True)
bass_track_real = get_track_transposed(midi_filename, 3, beat_resolution=beat_resolution, transpose=True)

soprano_pianoroll = midi_utils.scale_pianoroll(midi_utils.flatten_one_hot_pianoroll(soprano_track.pianoroll), global_lower)
alto_pianoroll_real = midi_utils.scale_pianoroll(midi_utils.flatten_one_hot_pianoroll(alto_track_real.pianoroll), global_lower)
tenor_pianoroll_real = midi_utils.scale_pianoroll(midi_utils.flatten_one_hot_pianoroll(tenor_track_real.pianoroll), global_lower)
bass_pianoroll_real = midi_utils.scale_pianoroll(midi_utils.flatten_one_hot_pianoroll(bass_track_real.pianoroll), global_lower)


# now we can predict the second voice 
alto_pianoroll = sample_harmonization(net_harmonization_alto, soprano_pianoroll, prime = alto_pianoroll_real[:4])

tenor_pianoroll = sample_harmonization(net_harmonization_tenor, soprano_pianoroll, prime = tenor_pianoroll_real[:4])
bass_pianoroll = sample_harmonization(net_harmonization_bass, soprano_pianoroll, prime = bass_pianoroll_real[:4])

# go back to the pitch range 
alto_pianoroll = midi_utils.unscale_pianoroll(alto_pianoroll, global_lower)
tenor_pianoroll = midi_utils.unscale_pianoroll(tenor_pianoroll, global_lower)
bass_pianoroll = midi_utils.unscale_pianoroll(bass_pianoroll, global_lower)

# convert to one-hot representation for track object
alto_onehot = midi_utils.one_hot_encode_pianoroll(alto_pianoroll, 128)*90
tenor_onehot = midi_utils.one_hot_encode_pianoroll(tenor_pianoroll, 128)*90
bass_onehot = midi_utils.one_hot_encode_pianoroll(bass_pianoroll, 128)*90

alto_track = Track(pianoroll=alto_onehot, name='alto gernerated track')
tenor_track = Track(pianoroll=tenor_onehot, name='tenor gernerated track')
bass_track = Track(pianoroll=bass_onehot, name='bass gernerated track')
# create a multitrack made of the initial soprano track and the generated alto
new_multitrack = Multitrack(tracks=[soprano_track, alto_track, tenor_track, bass_track], tempo = 90, beat_resolution=beat_resolution)

multitrack_real = Multitrack(tracks=[soprano_track, alto_track_real, tenor_track_real, bass_track_real], tempo = 90, beat_resolution=beat_resolution)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_harmonization_track")
pypianoroll.write(multitrack_real, home_dir + "/results/RNN_harmonization_track_real")

In [263]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename, 'test_fake.wav')
Audio("test_fake.wav")

Output hidden; open in https://colab.research.google.com to view.

In [262]:
modified_midi_filename_real = home_dir + "/results/RNN_harmonization_track_real.mid"
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename_real, 'test_real.wav')
Audio("test_real.wav")

Output hidden; open in https://colab.research.google.com to view.

In [0]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

error: ignored

In [0]:
pygame.mixer.music.stop()

error: ignored